# DATA DISCOVERY

The data discovery process involves the collection and evaluation of data from various sources. 
Thus, for the purpose of this case study, I have selected two different datasets from the kaggle repository.

**DATASET I - WORLD FOOD/FEED PRODUCTION**

The Food Balance Sheets FAO's dataset presents a comprehensive picture of the pattern of a country's food supply for over 245 countries and territories, from the year 1961 to 2013. The food balance sheet shows for each food item the sources of supply and its utilization.

**DATASET II - ENVIRONMENT IMPACT OF FOOD PRODUCTION**

The Food Production dataset contains 43 most common foods grown across the globe and 23 columns as their respective land, water usage and carbon footprints. In order words, it represents greenhouse gas emissions per kg of food product(Kg CO2 - equivalents per kg product) across different stages in the lifecycle of food production.

The information retrieved from these two datasets as well as their proper combination will let me build a clear and detailed picture on topics I have decided to cover with this study case. 

Let's import several helpful analytics libraries.
These libraries will let me upload, transform, analyse and inspect the chosen datasets.

In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
from collections import Counter

pd.options.display.max_columns = None

In [ ]:
import step1_data_selection.py
import step2_data_cleaning.py
import step3_data_exploration.py
import step4_data_trasformation.py

# DATA VISUALIZATION

After having accomplished all previous steps and have learned the datasets in detail, I can finally make the content visual. Thus, I will try to reply to the questions listed in the introduction section by ploting graphs and maps.

**What are the fastest growing countries in production terms?**

In order to reply to this question, I will refactor a little bit the ffp dataframe.
Namely, I will trasform the 53 columns referring to years ranging from 1961 to 2013 into two columns holding the year and its relative value.

In [ ]:
years = pd.Series(np.arange(1961, 2014, 1))
columns_name = pd.Series(['Area Abbreviation', 'Area', 'Item', 'Element', 'latitude', 'longitude'])
columns_name = pd.concat([columns_name, years])
columns_name

df = ffp
df.columns = columns_name
df = df.melt(id_vars=['Area Abbreviation', 'Area', 'Item', 'Element', 'latitude', 'longitude'], value_vars=years,
        var_name='Year', value_name='Production (kilo tonnes)')

df.head()

In [ ]:
aggs = ["sum","avg","stddev","max"]

agg = []
agg_func = []
for i in range(0, len(aggs)):
    agg = dict(
        args=['transforms[0].aggregations[0].func', aggs[i]],
        label=aggs[i],
        method='restyle'
    )
    agg_func.append(agg)

data = [dict(
  type = 'choropleth',
  locationmode = 'country names',
  locations = df['Area'],
  z = df['Production (kilo tonnes)'],
  autocolorscale = False,
  colorscale = 'Portland',
  reversescale = True,
  transforms = [dict(
    type = 'aggregate',
    groups = df['Area'],
    aggregations = [dict(
        target = 'z', func = 'sum', enabled = True)
    ]
  )]
)]

layout = dict(
  title = '<b>Total Food/Feed produced over the years by each country</b><br>use dropdown to change aggregation',
  height = 500,
  width = 700,
  updatemenus = [dict(
        x = 0.85,
        y = 1.15,
        xref = 'paper',
        yref = 'paper',
        yanchor = 'top',
        active = 1,
        showactive = False,
        buttons = agg_func
  )]
)

fig_dict = dict(data=data, layout=layout)

pio.show(fig_dict, validate=False)

The above map tells us the following facts:
* the top five food/feed producers are China, United States of America, India, Brazil and Germany.
* the five countries that mantained the highest average production over the years are still China, United States of America, India, Brazil and Germany.
* the five countries that managed to produce a record amount of food/feed in these years are China, India, United States of America, Russian Federation and Indonesia.
* the top five countries that were able to increase significantly their production over the years (high stddev) are China, United States of America, India,  

Some statements listed in the previous observation, are confirmed also by the below graph which displays the general production "average" and "stddev" trends. Thus, we can affirm that in these 50 years the production have constantly increased mainly because some countries were able to considerably intensify cultivation and land use for food/feed production (thus increase their stddev). 

In [ ]:
tt = ffp_qtv.T

fig, axs = plt.subplots(1, 2, figsize=(10,4))  

tt.plot.line( y = ['mean'], ax=axs[0])
tt.plot.line( y = ['std'], ax=axs[1])

fig.suptitle('Some statistical evaluation of ffp dataset content',fontsize = 14, fontweight = "bold")
plt.ylabel('Food/feed items produced in 1000 tonne',fontsize = 10)

plt.show()


Now let's have a look at the production rate of each country over the years in order to become aware of each country production trend.
For this purpose, I will refer to the "area_sum_ffp" dataframe which holds the estimations of the countries' total year production.

In [ ]:
sliced_sum_ffp = area_sum_ffp.loc[:,"Y1961":"Y2013"]

# Since I want to show how total production varies across the years, 
# I need to traspose the dataframe so that I get the years on "x-axes"
sliced_sum_ffp = sliced_sum_ffp.T

# I am replacing index values type by changing strings "Yxxxx" into numbers "xxxx".
# Change applied for plotting reasons
years = np.arange(1961, 2014, 1)
sliced_sum_ffp['Year'] = years 
sliced_sum_ffp = sliced_sum_ffp.set_index('Year')

Now I am going to use a line plot to represent the computed values.

In [ ]:
# You typically want your plot to be ~1.33x wider than tall. This plot
# is a rare exception because of the number of lines being plotted on it.
# Common sizes: (10, 7.5) and (12, 9)
fig, ax1 = plt.subplots(1,1,figsize=(12, 14))

# These are the colors that will be used in the plot
ax1.set_prop_cycle(color=[
    '#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c', '#98df8a',
    '#d62728', '#ff9896', '#9467bd', '#c5b0d5', '#8c564b', '#c49c94',
    '#e377c2', '#f7b6d2', '#7f7f7f', '#c7c7c7', '#bcbd22', '#dbdb8d',
    '#17becf', '#9edae5'])

# Remove the plot frame lines. They are unnecessary here.
ax1.spines[:].set_visible(False)

# Ensure that the axis ticks only show up on the bottom and left of the plot.
ax1.xaxis.tick_bottom()
ax1.yaxis.tick_left()

fig.subplots_adjust(left=.06, right=.75, bottom=.02, top=.94)
# Limit the range of the plot to only where the data is.
ax1.set_xlim(1960.5, 2013.5)
ax1.set_ylim(-0,25, 3250000)

# Set a fixed location and format for ticks.
ax1.set_xticks(range(1961, 2013, 10))
ax1.set_yticks(range(0, 3500000, 250000))

# Use automatic StrMethodFormatter creation
ax1.xaxis.set_major_formatter('{x:.0f}')
ax1.yaxis.set_major_formatter('{x:.0f}')

# Provide tick lines across the plot to help your viewers trace along
# the axis ticks. Make sure that the lines are light and small so they
# don't obscure the primary data lines.
ax1.grid(True, 'major', 'y', ls='--', lw=.5, c='k', alpha=.3)

# Remove the tick marks; they are unnecessary with the tick lines we just
# plotted. Make sure your axis ticks are large enough to be easily read.
# You don't want your viewers squinting to read your plot.
ax1.tick_params(axis='both', which='both', labelsize=14,
               bottom=False, top=False, labelbottom=True,
               left=False, right=False, labelleft=True)

# Now that the plot is prepared, it's time to actually plot the data!
majors = sliced_sum_ffp.columns

for column in majors:
    # Plot each line separately with its own color.
    line, = ax1.plot(sliced_sum_ffp.index, column, data=sliced_sum_ffp)

    # Add a text label to the right end of every line. Most of the code below
    # is adding specific offsets y position because some labels overlapped.
    y_pos = np.array(sliced_sum_ffp[column])[-1] - 0.5
    
    text = column + ' : ' + str(totals_ffp.loc[column, 'TotalProduction_%']) + '%'
    # Make sure that all labels are large enough to be easily read by the viewer.
    ax1.text(2013.5, y_pos, text, fontsize=12, color=line.get_color())

# The title 
fig.suptitle("Total Food/Feed produced each year by each country", fontsize=18, ha="center")

plt.show()

The above graph tells us the following things:

* China alone produces 20% of the global food/feed production and its production rate has been exponential and unstoppable. 
* From around 2005 India has overcomed USA and became the second global food/feed producer.
* From the moment Russian Federation became a nation, its production has been of around 2% (375.000 kilo tonnes)
* Brazil, Nigeria and Indonesia have costantly increased their production over the years, whereas Germany has decreased its production after the Fall of the Berlin Wall (1989) and kept it stable afterwards at around 200.000 kilo tonnes per year.


**What are the differences between food and feed production?**

In order to show the decomposition of production type among years and countries, I will use Matplotlib and Seaborn library functions.

In [ ]:
# I am adding the "Diff" column to "element_mean_ffp" dataframe
mean_ffp = element_mean_ffp.loc[:, "Y1961":"Y2013"]
diff = -mean_ffp.diff()
mean_ffp.loc[-1] = diff.loc["Food"]
mean_ffp.index = ["Feed", "Food", "Diff"]
mean_ffp = mean_ffp.T

# I am replacing index values type by changing strings "Yxxxx" into numbers "xxxx".
years = np.arange(1961, 2014, 1)
mean_ffp['Year'] = years 
mean_ffp = mean_ffp.set_index('Year')

# I am dropping the "Food" and "Feed" columns since I do not need them any more
mean_ffp = mean_ffp.drop(columns = ["Feed", "Food"])

In [ ]:
plt.figure()


(df.pivot_table(
    index="Year",
    columns="Element",
    values="Production (kilo tonnes)",
    aggfunc='mean').plot(kind='bar', figsize=(20, 8), title = "Average Food/Feed produced each year")
)

mean_ffp.plot(kind = "line", figsize=(20, 8), title = "Difference between feed and food production", legend = False)

plt.show()

The above representations tell us that the feed production always exceeds the food production, with the highest difference among them being registered in years 1992:1994 (possible reason: Russian Federation inserted into the countries list). Moreover, it is noticeable also here that the production trend is ascending and increasing almost constantly.

Let's now give a look at the proportion of feed and food each country produced over around 50 years.

In [ ]:
mean_ffp = general_mean_ffp.T
mean_ffp = mean_ffp.reset_index()
mean_ffp.columns = ["Area", "Element", "Value"]
mean_ffp = mean_ffp.sort_values(by = "Value", ascending = False)


In [ ]:
fig = px.sunburst(mean_ffp, path=['Element', 'Area'], values="Value", title='Countries food and feed production ranking')
fig.show()

In [ ]:
fig, axs = plt.subplots(figsize = (15, 40))

sns.barplot(x="Value", y="Area", hue="Element", data=mean_ffp, palette="coolwarm")

# The title 
plt.title("Total Food/Feed produced by each country", fontsize=14)

plt.show()

The above graphs show us the proportion of food (or feed) produced by each country in all these years.
In the first graph is more explicit which countries contribute most to the production of each element.
Whereas, in the second graph is more explicit on which type of element each country bases its production .

**How diversified is the food/feed offer?**

The below graph shows us that the food production is much more diversified than the feed production since it has around x4 times more product items.

In [ ]:
fig = px.histogram(ffp_qlt, x="Area", color="Element", title='Histogram of food/feed variety per country', opacity=0.8)
fig.show()

Now, let's check of what kind of items each category (food/feed) is composed and how much each item weights with respect to other items in a certain category.

In [ ]:
ff = ffp_qlt.groupby(["Element", "Item"]).count()
ff = ff.reset_index()
ff = ff.drop(columns = "Area Abbreviation")
#ff.head()

fig = px.sunburst(ff, path=['Element', 'Item'], values="Area", title='Food and Feed items and their relevance')
fig.show()

From the above graph we extrapolate the following things:

* Milk is one of the most required and produced item both as food and feed product.
* The top ten food items are: eggs, milk, animal fats, apples, bovine meat, butter, cereals, cocoa beans, coffee, fish/seafood.
* The top ten feed items are: milk, cereals, fish/seafood, maize, pelagic fish, oilcrops, cereals other, starchy roots, sorghum, oilcrops others.

If we ignore the categorization of products and  give a closer look at the type of products produced on global scale, we would note that the top five most produced items are milk, eggs, cereals, fish/seafood and maize.  

In [ ]:
pp = ffp_qlt["Item"].value_counts(ascending = True)

# define figure
fig, ax = plt.subplots(1, figsize=(15, 22))
# numerical x
x = np.arange(0, len(pp.index))

# plot horizontal bars
ax = pp.plot.barh()

# x y details
plt.yticks(ticks=range(len(pp.index)), labels=pp.index)

# grid lines
ax.set_axisbelow(True)
ax.xaxis.grid(color='gray', linestyle='dashed', alpha=0.2)

# title and legend
plt.title('Items global production ranking', fontsize = 14)

plt.show()

**Which types of food have more negative impact on the environment?**

First, I am going to give evidence of total carbon footprint trend among different product types.

In [ ]:
fpi= fpi.sort_values(['TotalEmissions'])

# define figure
fig, ax = plt.subplots(1, figsize=(20, 13))
# numerical x
x = np.arange(0, len(fpi.index))

# plot horizontal bars
plt.barh(x, fpi['TotalEmissions'])

# x y details
plt.xlabel('KG CO2 x KG Food',fontsize = 12)
plt.yticks(ticks=range(len(fpi.index)), labels=fpi.index)

# grid lines
ax.set_axisbelow(True)
ax.xaxis.grid(color='gray', linestyle='dashed', alpha=0.2)

# title and legend
plt.title('Carbon Footprint per food product', fontsize = 18)

plt.show()

By looking at this graph, we notice that there are some food products in this dataset who have a total emission of more than 15 Kg CO2 per kg of produced product, such as: **beef, lamb & mutton, cheese, dark chocolate and coffee**. Thus, they can be considered as the products with the highest carbon footprint.

Just for completeness, I am going to analyse the "TotalEmissions" column data from statistical point of view.

In [ ]:
sns.set_theme(style="white")
sns.set_style("ticks")

plt.figure(figsize=(18,6))

plt.subplot(131)

box = sns.boxplot(y = "TotalEmissions", data=fpi, orient = "v", linewidth=0.75)
#box = sns.swarmplot(y = "TotalEmissions", data=fpi, color=".25")

box.set_ylabel("Kg CO2 x KG Product",fontsize = 10)
box.set_title("BOX PLOT (Total Emissions)")

plt.subplot(132)

hist = sns.histplot(fpi['TotalEmissions'], bins = 20)

hist.set_title("HISTOGRAM (Total Emissions)")
hist.set_xlabel("KG CO2 x KG Product",fontsize = 10)
hist.set_ylabel("Frequency",fontsize = 10)

plt.subplot(133)

kde = sns.kdeplot(fpi['TotalEmissions'],
                   shade = True,
                   color = "b")

kde.set_title("KDE PLOT (Total Emissions)")
kde.set_xlabel("Kg CO2 x KG Product",fontsize = 10)
kde.set_ylabel("Density",fontsize = 10)

plt.show()


The diplayed box plot shows that:

* There are lines extending on either side of the box plot on either side. They tell us that some data exists a little off from the max concentration. In boxplot terms, the whiskers extend to points that lie within 1.5 times the interquartile range of the lower and upper quartile. Observations falling outside this margin are displayed independently.
* Most emissions are between 0,85 to 6 KG CO2 emission on an average per KG of produced food.
* The average number of emission is around 1,6 KG CO2 emission , shown by the box middle line.
* Some food products in this dataset have a total emission of more than 15 Kg CO2 per kg product.


The other two functions, displayed next to the box plot, show the Histogram and Kernel Density Estimation curve of total emissions data. From the following operation on the 'TotalEmissions'column (contains the calculated total emission in Kg CO2 per given food product), I can see that most of the time, total emissions value is in the range 0-0.6 and from the kde curve it can be seen that the most frequently observed value is about 0.3.

**How much each stage of food production contributes to the total emission?**

Let's see in more detail by what percentage each step inside the food production process affects the amount of emission registered per single food type.

In [ ]:
# Fuction that normalize the data by converting values in range [0,1]
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

# Define the colors palette to be used on pie charts
colors = sns.color_palette('pastel')[0:7]

# Workaround to insert labels into a LEGEND and not on the pie chart (useful to avoid text overlapping)
labels = ['LandUseChange','AnimalFeed','Farm','Processing','Transport','Packaging','Retail']
unit = ['Kg CO2']*7
legend_labels = ['{0} ({1})'.format(i,j) for i,j in zip(labels,unit)]

fig = plt.figure(figsize = (60,320))

# Iterates over row index (=type of food product)
count = 0

for row_index, row in fpi.iterrows():
    
    count +=1
    
    data = NormalizeData(row[0:7])
    
    ax = fig.add_subplot(15,3,count)
    
    wedges, label, autopct = ax.pie(data, colors = colors, autopct = '%0.0f%%', startangle=90)

    plt.setp(autopct, size=42)
    plt.setp(label, size=0)

    # Display the LEGEND every 3 pie charts
    if ((count-1)%3==0):
        plt.legend(wedges,legend_labels,loc='upper left',bbox_to_anchor=(-0.65, 1.),fontsize=40, title="LEGEND", title_fontsize=44)

    ax.set_title(row_index,fontsize = 48, fontweight = "bold")

**How much each food type contributes to Eutrophying and GreenhouseGas emissions?**

I will create another dataframe (a sliced version of fpi dataframe), which will contain only the fpi columns referred to Eutrophying and GreenhouseGas emissions.

In [ ]:
# loading of a dataframe from seaborn
df = fpi.loc[:, 'EutrophyingEmissions_kg' : 'ScarcityWeightedWaterUse_kg']
df.stack()  

In [ ]:
# define figure
fig, ax = plt.subplots(1, figsize=(15, 10))
# numerical x
x = np.arange(0, len(df.index))
# plot bars
plt.bar(x - 0.2, df['EutrophyingEmissions_kg'])
plt.bar(x + 0.0, df['GreenhouseGasEmissions_kg'])

# remove spines
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
# x y details
plt.ylabel('Amount of emission')
plt.xticks(ticks=range(len(df.index)), labels=df.index, rotation=90 )
plt.xlim(-0.5, 45)
# grid lines
ax.set_axisbelow(True)
ax.yaxis.grid(color='gray', linestyle='dashed', alpha=0.2)
# title and legend
plt.title('Emission per Kg of food production', fontsize = 18)
plt.legend(['EutrophyingEmissions:gPO₄eq x Kg', 'GreenhouseGasEmissions:kgCO₂eq x 1000Kcal'], loc='upper left', ncol = 2)
plt.show()

The above graph, tells us that there is little relationship between Eutrophying and GreenhouseGas emission. In other words, we notice that, when one of these measures increases it does not affect proportionally the other measure. In fact we observe the following:
* Greehouse Gas emission is unrelated to other measurements and is caused mainly by these few food types: coffee, beed herds,lamb & mutton and tomatoes.
* Eutrophying emission is very high for beef and fish.

**What is the impact of food production on other resources (water, land)?**

Let's first give a look at the water withdrawals for production purposes. 

Note that the scarcity-weighted water use indicator represents freshwater use weighted by local water scarcity.

In [ ]:
# define figure
fig, ax = plt.subplots(1, figsize=(20, 13))
# numerical x
x = np.arange(0, len(df.index))

# plot horizontal bars
plt.barh(x, df['ScarcityWeightedWaterUse_kg'])
plt.barh(x, df['FreshwaterWithdrawals_kg'], left = df['ScarcityWeightedWaterUse_kg'])

# x y details
plt.xlabel('Amount of used water')
plt.yticks(ticks=range(len(df.index)), labels=df.index)
plt.ylim(-0.5, 45)

# grid lines
ax.set_axisbelow(True)
ax.xaxis.grid(color='gray', linestyle='dashed', alpha=0.2)

# title and legend
plt.title('Water used per Kg of food production', fontsize = 18)
plt.legend(['ScarcityWeightedWaterUse','FreshwaterWithdrawals'], loc='upper right', ncol = 2)
plt.show()

The weightning of the scarcity-weighted water use indicator looks to be quite aggressive and not proportional to the amount of freshwater use. Instead it gives much more relevance to local water scarcity indicator. In fact, many of the bars having similar extend of the red bar portion (freshwater use), has a very different extend of the blu bar portion (scarcity-weighted water use).

Anyway, by looking at this plot, we can observe that:

* the food products that require a lot of freshwater are: cheese, nuts, fish and beef (dairy herd)
* the food products with the highest water use footprint are: nuts, cheese, olive oil, lamb & mutton, beef (dairy herd)

Now let's give a look at the land use parameter.

In [ ]:
# define figure
fig, ax = plt.subplots(figsize=(15, 10))
# numerical x
x = np.arange(0, len(df.index))
# plot line
ax.plot(x, df['LandUse_kg'])

# remove spines
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
# x y details
plt.ylabel('Land Use')
plt.xticks(ticks=range(len(df.index)), labels=df.index, rotation=90 )
# grid lines
ax.set_axisbelow(True)
ax.yaxis.grid(color='gray', linestyle='dashed')
# title and legend
plt.title('Land Use per each kg of produced food', fontsize=14)

plt.show()

By looking at this graph, we notice that there are some food products who requires lots of land, such as: beef, lamb & mutton, cheese, dark chocolate and coffee. Thus, they can be considered as the products with the highest land use footprint.